## 逻辑回归 LR

逻辑回归（Logistic Regression）主要解决二分类问题，用来**表示某件事情发生的可能性**
- 某邮件是垃圾邮件的可能性（是、不是）
- 购买某商品的可能性（买、不买）
- 某广告被点击的可能性（点、不点）

- 优点：易于理解，可并行化，可解释性强
- 缺点：容易欠拟合，一般准确度不太高

### sigmod 函数
<img src="https://cdn.jsdelivr.net/gh/MatNoble/Images/20210321200201.png"/>

### 原理
  Logistic 回归的本质是：假设数据服从 Logistic 分布，然后使用「极大似然估计」做参数的估计  
  **分布函数**
  $$
  F(x) = P(X \leq x) = \frac{1}{1+e^{-(x-\mu)/\gamma}}
  $$
  **密度函数**
  $$
  f(x) = F'(x) = \frac{e^{-(x-\mu)/\gamma}}{\gamma(1+e^{-(x-\mu)/\gamma})^2}
  $$
  类似于正态分布
### 模型
  $$
  P(Y=1|x) = y = \frac{1}{1+e^{-(w^Tx+b)}}
  $$
  输出的是样本为正的概率
  $$
  \ln \frac{P(Y=1|x)}{P(Y=0|x)} = \ln \frac{y}{1-y} = w^Tx+b = \tilde{w} \cdot \tilde{x}
  $$
### 损失函数
- **似然函数**  
  设 $p(x) = P(Y=1|x)$, 则
  $$
  \Pi_{i=1}^n [p(x_i)]^{y_i} [1-p(x_i)]^{1-y_i}
  $$
  取对数，「乘法 $\to$ 加法」
  $$
  \begin{aligned}
  \max ~ J(w) &= \sum_{i=1}^n \bigl[y_i \ln p(x_i) + (1-y_i)\ln (1-p(x_i)) \bigr] \\
  & = \sum_{i=1}^n \bigl[ y_i \ln \frac{p(x_i)}{1-p(x_i)} + \ln(1-p(x_i)) \bigr] \\
  & = \sum_{i=1}^n \bigl[ y_i (w \cdot x_i) - \ln (1+\exp(w \cdot x_i)) \bigr]
  \end{aligned}
  $$
- **交叉熵损失函数**  
  **最大化似然函数** $\Longleftrightarrow$ **最小化损失函数**
  $$
  \min~ L(w) = \min~ -\frac{1}{n} J(w)
  $$
### 求解
  - 一阶 **梯度下降**
  $$
  w^{k+1}_i = w^k_i - \alpha g_i
  $$
  其中
  $$
  g_i = \frac{\partial J(w)}{\partial w_i} = (p(x_i)-y_i)^T x_i
  $$
  - 二阶 **牛顿法**
  $$
  w^{k+1} = w^k - \frac{J'(w)}{J''(w)} = w^k - H^{-1}g^k
  $$

### 并行化
  对目标函数**梯度计算**的并行化
  - 不同的数据
  - 不同的特征

### 应用
预测用户对品类的购买偏好

### 逻辑回归为什么要对特征进行离散化？
① 非线性！非线性！非线性！逻辑回归属于广义线性模型，表达能力受限；单变量离散化为N个后，每个变量有单独的权重，相当于为模型引入了非线性，能够提升模型表达能力，加大拟合； 离散特征的增加和减少都很容易，易于模型的快速迭代；

② 速度快！速度快！速度快！稀疏向量内积乘法运算速度快，计算结果方便存储，容易扩展；

③ 鲁棒性！鲁棒性！鲁棒性！离散化后的特征对异常数据有很强的鲁棒性：比如一个特征是年龄>30是1，否则0。如果特征没有离散化，一个异常数据“年龄300岁”会给模型造成很大的干扰；

④ 方便交叉与特征组合：离散化后可以进行特征交叉，由M+N个变量变为M*N个变量，进一步引入非线性，提升表达能力；

⑤ 稳定性：特征离散化后，模型会更稳定，比如如果对用户年龄离散化，20-30作为一个区间，不会因为一个用户年龄长了一岁就变成一个完全不同的人。当然处于区间相邻处的样本会刚好相反，所以怎么划分区间是门学问；

⑥ 简化模型：特征离散化以后，起到了简化了逻辑回归模型的作用，降低了模型过拟合的风险。


### 与其他模型对比

- **线性回归**
  - 逻辑回归基于线性回归，处理分类问题
  - 增加 sigmod 函数非线性映射
  
  **Sigmoid 函数作用：**
  1. $\mathbb{R} \to [0, 1]$ 非线性映射，概率化
  2. 线性回归在实数域上敏感度一致，而逻辑回归在 0 附近敏感，在远离 0 点位置不敏感，这个的好处就是模型更加「关注分类边界」，可以**增加模型的鲁棒性**

- **最大熵模型**
  逻辑回归和最大熵模型本质上没有区别，最大熵在解决二分类问题时就是逻辑回归，在解决多分类问题时就是多项逻辑回归。

- **SVM**
  - 联系
    1. 都处理分类问题
    2. 都可以加正则化，结果近似
  - 区别
    1. LR 是参数模型，SVM 是非参数模型。
    2. 从目标函数来看，区别在于逻辑回归采用的是Logistical Loss，SVM采用的是hinge loss，这两个损失函数的目的都是**增加对分类影响较大的数据点的权重，减少与分类关系较小的数据点的权重**
    3. SVM 的处理方法是只考虑 Support Vectors，也就是和分类最相关的少数点，去学习分类器。而逻辑回归通过**非线性映射**，大大减小了离分类平面较远的点的权重，相对提升了与分类最相关的数据点的权重。
    4. 逻辑回归相对来说模型**更简单，好理解**，特别是大规模线性分类时比较方便。而SVM的理解和优化相对来说复杂一些，SVM 转化为**对偶问题**后,分类只需要计算与少数几个支持向量的距离,这个在进行**复杂核函数**计算时优势很明显,能够大大简化模型和计算。
    5. Logic 准确率更高，SVM 使用更广(Logic 能做的 SVM 能做，但可能在准确率上有问题，SVM 能做的 Logic 有的做不了。)